# MATH&ML-2. Линейная алгебра в контексте линейных методов. Часть II

In [515]:
# Загрузка библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
import seaborn as sns # для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

In [516]:
# Задание 1.1
v1 = np.array([-1, 2, -7, 9]).T
v2 = np.array([2, 8, 2, -1]).T

print(v1@v2)
print('-'*50)

#Задание 1.4
A = np.array([[1,1], [1,2]])
print(A.T@A)
print('-'*50)

#Задание 1.6
A = np.array([[1,2], [3,7]])
print(np.linalg.inv(A))
print('-'*50)

#Задание 1.7
v1 = np.array([2, 10, -2]).T
v2 = np.array([3, 2, -2]).T
v3 = np.array([8, 14, -6]).T

A = np.array([v1, v2, v3])
print(np.linalg.matrix_rank(A))
print(A)

-9
--------------------------------------------------
[[2 3]
 [3 5]]
--------------------------------------------------
[[ 7. -2.]
 [-3.  1.]]
--------------------------------------------------
2
[[ 2 10 -2]
 [ 3  2 -2]
 [ 8 14 -6]]


In [517]:
print(np.array([[1,2],[-3,1],[1,2],[1,-1]]))
print(np.array([[1,2],[-3,1],[1,2],[1,-1]]).T)

[[ 1  2]
 [-3  1]
 [ 1  2]
 [ 1 -1]]
[[ 1 -3  1  1]
 [ 2  1  2 -1]]


In [518]:
#Задание 2.11

v1 = [1,-3,1,1]
v2 = [2,1,2,-1]
b = [1,4,5,0]

A = np.array([v1,v2]).T
gramm_A = A.T@A
print('Матрица Грамма для A:\n', gramm_A)
AtA_inv = np.linalg.inv(gramm_A)
print('Обратная матрица грамма:\n', AtA_inv)
Atb = A.T@b
print('A.T@b:\n', Atb)
print('Вектор коэффициентов w:', AtA_inv@Atb)

Матрица Грамма для A:
 [[12  0]
 [ 0 10]]
Обратная матрица грамма:
 [[0.08333333 0.        ]
 [0.         0.1       ]]
A.T@b:
 [-6 16]
Вектор коэффициентов w: [-0.5  1.6]


In [519]:
# загружаем датасет
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
boston_data = pd.read_csv('data/housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()

# составляем матрицу А и вектор целевой переменной
CRIM = boston_data['CRIM']
RM = boston_data['RM']
A = np.column_stack((np.ones(506), CRIM, RM))
y = boston_data[['PRICE']]
print(A)
print('Размерность:', A.shape)
w_hat = np.linalg.inv(A.T@A)@A.T@y
print(w_hat.values)

# добавились новые данные:
CRIM_new = 0.1
RM_new = 8
# делаем прогноз типичной стоимости дома
PRICE_new = w_hat.iloc[0]+w_hat.iloc[1]*CRIM_new+w_hat.iloc[2]*RM_new
print('Прогнозная цена на новых данных:',
      PRICE_new.values)
# короткий способ сделать прогноз
new=np.array([[1,CRIM_new,RM_new]])
print('prediction:', (new@w_hat).values)

# Задание 3.5
CRIM_35 = 0.2
RM_35 = 6
new = np.array([[1,CRIM_35, RM_35]])
print('Prediction in 3.5:', (new@w_hat).values)

#Задание 3.7
y = [1,2,1,2]
x1 = [1,1,0,2]
x2 = [-1,1,0,0]
x3 = [0,2,0,2]

A = np.array([x1,x2,x3])
# w_hat=np.linalg.inv(A.T@A)@A.T@y
# Выдаст ошибку LinAlgError: матрица вырождена

# создадим вырожденную матрицу А
A = np.array([
    [1, 1, 1, 1], 
    [2, 1, 1, 2], 
    [-2, -1, -1, -2]]
).T
y = np.array([1, 2, 5, 1])

model = LinearRegression(fit_intercept=False)
model.fit(A, y)
print('w_hat:', model.coef_)

[[1.0000e+00 6.3200e-03 6.5750e+00]
 [1.0000e+00 2.7310e-02 6.4210e+00]
 [1.0000e+00 2.7290e-02 7.1850e+00]
 ...
 [1.0000e+00 6.0760e-02 6.9760e+00]
 [1.0000e+00 1.0959e-01 6.7940e+00]
 [1.0000e+00 4.7410e-02 6.0300e+00]]
Размерность: (506, 3)
[[-29.24471945]
 [ -0.26491325]
 [  8.39106825]]
Прогнозная цена на новых данных: [37.85733519]
prediction: [[37.85733519]]
Prediction in 3.5: [[21.04870738]]
w_hat: [ 6.   -1.25  1.25]


### Юнит 3

In [520]:
display(boston_data[['CHAS', 'LSTAT', 'CRIM','RM']].describe())

# составляем матрицу наблюдений и вектор целевой переменной
A = np.column_stack((np.ones(506), boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]))
y = boston_data[['PRICE']]
# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat=np.linalg.inv(A.T@A)@A.T@y
print(w_hat.values)

# составляем матрицу наблюдений без дополнительного столбца из единиц
A = boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]
y = boston_data[['PRICE']]
# стандартизируем векторы в столбцах матрицы A
A_cent = A - A.mean(axis=0)
A_st = A_cent/np.linalg.norm(A_cent, axis=0)
display(A_st.describe().round(2))

,CHAS,LSTAT,CRIM,RM
count,506.000000,506.000000,506.000000,506.000000
mean,0.069170,12.653063,3.613524,6.284634
std,0.253994,7.141062,8.601545,0.702617
min,0.000000,1.730000,0.006320,3.561000
25%,0.000000,6.950000,0.082045,5.885500
50%,0.000000,11.360000,0.256510,6.208500
75%,0.000000,16.955000,3.677083,6.623500
max,1.000000,37.970000,88.976200,8.780000


[[-1.92052548]
 [ 3.9975594 ]
 [-0.58240212]
 [-0.09739445]
 [ 5.07554248]]


,CHAS,LSTAT,CRIM,RM
count,506.00,506.00,506.00,506.00
mean,-0.00,-0.00,-0.00,-0.00
std,0.04,0.04,0.04,0.04
min,-0.01,-0.07,-0.02,-0.17
25%,-0.01,-0.04,-0.02,-0.03
50%,-0.01,-0.01,-0.02,-0.00
75%,-0.01,0.03,0.00,0.02
max,0.16,0.16,0.44,0.16


In [521]:
# Задание 4.3

x = np.array([12,8]).T
display(x)
x_cent = x-x.mean(axis=0)
x_std = x_cent/np.linalg.norm(x_cent, axis=0)
np.round(x_std, 3)

# Задание 4.7

print('-'*50)

v = np.array([5,1,2])
u = np.array([4,2,8])
print(np.round(np.corrcoef(v, u), 2))

# Задание 4.8

print('-'*50)

x_1 = np.array([5.1, 1.8, 2.1, 10.3, 12.1, 12.6])
x_2 = np.array([10.2, 3.7, 4.1, 20.5, 24.2, 24.1])
x_3 = np.array([2.5, 0.9, 1.1, 5.1, 6.1, 6.3])
C = np.corrcoef([x_1, x_2, x_3])
print('Rank:', np.linalg.matrix_rank(C))
print('Determinant: {:.7f}'.format(np.linalg.det(C)))
print(C)


array([12,  8])

--------------------------------------------------
[[1.   0.05]
 [0.05 1.  ]]
--------------------------------------------------
Rank: 3
Determinant: 0.0000005
[[1.         0.99925473 0.99983661]
 [0.99925473 1.         0.99906626]
 [0.99983661 0.99906626 1.        ]]


### Юнит 6. Полиномия

In [522]:
# Пример № 1: Построить квадратичную регрессию на целевую переменную y из одного фактора x1

A = np.array([
    [1, 1, 1, 1],
    [1, 3, -2, 1],
    [1, 9, 4, 1]
]).T
y = np.array([4, 5, 2, 2])
w_hat = np.linalg.inv(A.T@A)@A.T@y
print(w_hat)

[2.4        0.46666667 0.13333333]


In [523]:
# Пример № 2: Построить квадратичную регрессию на целевую переменную y из двух факторов x1 и x2

A = np.array([
    [1, 1, 1, 1, 1, 1, 1],
    [1, 3, -2, 1, 5, 13, 1],
    [3, 4, 5, -2, 4, 11, 3],
    [1, 9, 4, 1, 25, 169, 1],
    [3, 12, -10, -2, 20, 143, 3],
    [9, 16, 25, 4, 16, 121, 9]
    
]).T
y = np.array([4, 5, 2, 2, 6, 8, -1])
w_hat = np.linalg.inv(A.T@A)@A.T@y
print(w_hat)

[-2.25799015  2.37672337 -0.1322068  -0.10208147 -0.26501791  0.29722471]


In [524]:
# Пример № 3: Строится полиномиальная регрессия второй степени, задано три фактора:

A = np.array([
    [1, 3, -2, 1, 5, 13, 1],
    [3, 4, 5, -2, 4, 11, 3],
    [4, 5, 2, 2, 6, 8, -1],
]).T
print(A)

poly = PolynomialFeatures(degree=2, include_bias=True)
A_poly = poly.fit_transform(A)
display(pd.DataFrame(A_poly))

[[ 1  3  4]
 [ 3  4  5]
 [-2  5  2]
 [ 1 -2  2]
 [ 5  4  6]
 [13 11  8]
 [ 1  3 -1]]


,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,3.0,4.0,1.0,3.0,4.0,9.0,12.0,16.0
1,1.0,3.0,4.0,5.0,9.0,12.0,15.0,16.0,20.0,25.0
2,1.0,-2.0,5.0,2.0,4.0,-10.0,-4.0,25.0,10.0,4.0
3,1.0,1.0,-2.0,2.0,1.0,-2.0,2.0,4.0,-4.0,4.0
4,1.0,5.0,4.0,6.0,25.0,20.0,30.0,16.0,24.0,36.0
5,1.0,13.0,11.0,8.0,169.0,143.0,104.0,121.0,88.0,64.0
6,1.0,1.0,3.0,-1.0,1.0,3.0,-1.0,9.0,-3.0,1.0


In [525]:
def polynomial_regression(X, y, k):
    poly = PolynomialFeatures(degree=k, include_bias=True)
    X_poly = poly.fit_transform(X)
    w_hat = np.linalg.inv(X_poly.T@X_poly)@X_poly.T@y
    y_pred = X_poly @ w_hat
    return X_poly, y_pred, w_hat

A = boston_data[['LSTAT', 'PTRATIO', 'RM', 'CRIM']]
y = boston_data[['PRICE']]
 
A_poly, y_pred, w_hat = polynomial_regression(A, y, 1)
A_poly2, y_pred2, w_hat2 = polynomial_regression(A, y, 2)
A_poly3, y_pred3, w_hat3 = polynomial_regression(A, y, 3)
A_poly4, y_pred4, w_hat4 = polynomial_regression(A, y, 4)
A_poly5, y_pred5, w_hat5 = polynomial_regression(A, y, 5)

print('MAPE для полинома 1-й степени {:.2f}%'.format(metrics.mean_absolute_percentage_error(y, y_pred)*100))
print('MAPE для полинома 2-й степени {:.2f}%'.format(metrics.mean_absolute_percentage_error(y, y_pred2)*100))
print('MAPE для полинома 3-й степени {:.2f}%'.format(metrics.mean_absolute_percentage_error(y, y_pred3)*100))
print('MAPE для полинома 4-й степени {:.2f}%'.format(metrics.mean_absolute_percentage_error(y, y_pred4)*100))
print('MAPE для полинома 5-й степени {:.2f}%'.format(metrics.mean_absolute_percentage_error(y, y_pred5)*100))

MAPE для полинома 1-й степени 18.20%
MAPE для полинома 2-й степени 13.41%
MAPE для полинома 3-й степени 12.93%
MAPE для полинома 4-й степени 10.74%
MAPE для полинома 5-й степени 405.28%


In [526]:
# считаем матрицу корреляций (без столбца из единиц)
C_5 = pd.DataFrame(A_poly5[:, 1:]).corr()
# считаем ранг корреляционной матрицы
print('5-я степень:')
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(C_5))
# считаем количество факторов (не включая столбец из единиц)
print('Количество факторов:', A_poly5[:, 1:].shape[1])
print('-'*20)
print('4-я степень:')
C_4 = pd.DataFrame(A_poly4[:, 1:]).corr()
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(C_4))
print('Количество факторов:', A_poly4[:, 1:].shape[1])

display(pd.DataFrame(w_hat5).describe())
display(pd.DataFrame(w_hat4).describe())

5-я степень:
Ранг корреляционной матрицы: 110
Количество факторов: 125
--------------------
4-я степень:
Ранг корреляционной матрицы: 69
Количество факторов: 69


,PRICE
count,126.000000
mean,777.476127
std,22744.733602
min,-115841.128874
25%,-1.130875
50%,-0.000714
75%,0.858174
max,223546.237697


,PRICE
count,70.000000
mean,-50.900930
std,887.464445
min,-6926.341496
25%,-0.187941
50%,-0.000787
75%,0.322225
max,2305.476714


In [527]:
def polynomial_regression_sk(X, y, k):
    poly = PolynomialFeatures(degree=k, include_bias=False)
    X_poly = poly.fit_transform(X)
    lr = LinearRegression().fit(X_poly, y)
    y_pred = lr.predict(X_poly)
    return X_poly, y_pred, lr.coef_

A = boston_data[['LSTAT', 'PTRATIO', 'RM', 'CRIM']]
y = boston_data[['PRICE']]

for k in range(1, 6):
    A_poly, y_pred, w_hat = polynomial_regression_sk(A, y, k)
    print(
        "MAPE для полинома степени {} — {:.2f}%, СКО — {:.0f}".format(
            k, metrics.mean_absolute_percentage_error(y, y_pred)*100, w_hat.std()
        )

    )

MAPE для полинома степени 1 — 18.20%, СКО — 2
MAPE для полинома степени 2 — 13.41%, СКО — 5
MAPE для полинома степени 3 — 12.93%, СКО — 9
MAPE для полинома степени 4 — 10.74%, СКО — 304
MAPE для полинома степени 5 — 9.02%, СКО — 17055


In [528]:
# задание 6.2
x_62 = np.array([1,3,-2,9])

x_62 = x_62.reshape(-1,1)
poly_62 = PolynomialFeatures(degree=2, include_bias=True)
A_poly_62 = poly_62.fit_transform(x_62)
display(pd.DataFrame(A_poly_62))

# Задание 6.4
y_64 = np.array([3,7,-5,21])

w_hat_64 = polynomial_regression(x_62, y_64, 2)[2]
print(np.round(w_hat_64, 1))

,0,1,2
0,1.0,1.0,1.0
1,1.0,3.0,9.0
2,1.0,-2.0,4.0
3,1.0,9.0,81.0


[ 0.1  2.5 -0. ]


### Юнит 7. Регуляризация

In [529]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]
 
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)
 
# создаём модель линейной регрессии
lr = LinearRegression()
 
# оцениваем качество модели на кросс-валидации, метрика — MAPE
cv_results = cross_validate(lr, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))	
cv_results = pd.DataFrame(cv_results)
cv_results

MAPE на тренировочных фолдах: 12.64 %
MAPE на валидационных фолдах: 24.16 %


,fit_time,score_time,test_score,train_score
0,0.006995,0.000999,-0.093433,-0.143479
1,0.001020,0.000981,-0.330403,-0.130910
2,0.002008,0.000000,-0.090492,-0.139774
3,0.005979,0.001001,-0.268700,-0.108901
4,0.000992,0.001000,-0.424775,-0.108753


In [530]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# получаем оценку коэффициентов регрессии по МНК (точнее, ошибку singular matrix)
# w_hat = np.linalg.inv(A.T@A)@A.T@y
# print(w_hat)

In [531]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# единичная матрица
E = np.eye(3)
# коэффициент регуляризации 
alpha = 5
# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
w_hat_ridge = np.linalg.inv(A.T@A+alpha*E)@A.T@y
print(w_hat_ridge)

# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
ridge = Ridge(alpha=5, fit_intercept=False)
ridge.fit(A, y)
print(ridge.coef_)

[0.6122449  0.29387755 0.5877551 ]
[0.6122449  0.29387755 0.5877551 ]


In [532]:
# Проверка действия регуляризации полиномов с помощью кросс-валидации
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]
# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)
# создаём модель линейной регрессии c L2-регуляризацией
ridge = Ridge(alpha=20, solver='svd')
# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(ridge, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 12.54 %
MAPE на валидационных фолдах: 17.02 %


In [533]:
# Задание 7.4
A_74 = np.array([
    [1,1,1,1,1],
    [5,9,4,3,5],
    [15,18,18,19,19],
    [7,6,7,7,7]
]).T
y_74 = [24,22,35,33,36]
# Без регуляризации
w_hat_74 = np.linalg.inv(A_74.T@A_74)@A_74.T@y_74
print('Без регуляризации:', np.round(w_hat_74, 2))
# L2 регуляризация
E_74 = np.eye(4)
alpha_74 = 1
w_hat_74_ridge = np.linalg.inv(A_74.T@A_74+alpha_74*E_74)@A_74.T@y_74
print('С регуляризацией:', np.round(w_hat_74_ridge, 2))

Без регуляризации: [-140.82    1.18    3.     16.36]
С регуляризацией: [-0.09 -1.71  1.91  0.73]


In [534]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# получаем оценку коэффициентов регрессии с помощью L1-регуляризации
lasso = Lasso(alpha=0.1, fit_intercept=False)
lasso.fit(A, y)
print(lasso.coef_)

print('-'*20)
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]

# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L1-регуляризацией
lasso = Lasso(alpha=0.1, max_iter=10000)

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(lasso, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean() * 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

[1.14925373 0.         0.71921642]
--------------------
MAPE на тренировочных фолдах: 12.44 %
MAPE на валидационных фолдах: 16.44 %


In [535]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# получаем оценку коэффициентов регрессии (первый случай)
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.2, fit_intercept=False)
elasticnet.fit(A, y)
print(elasticnet.coef_)
# получаем оценку коэффициентов регрессии (второй случай)
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.7, fit_intercept=False)
elasticnet.fit(A, y)
print(elasticnet.coef_)
# получаем оценку коэффициентов регрессии (третий случай)
elasticnet = ElasticNet(alpha=0.1, l1_ratio=1, fit_intercept=False)
elasticnet.fit(A, y)
print(elasticnet.coef_)
print('-'*30)

# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]
# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)
# создаём модель линейной регрессии c L1- и L2-регуляризациями
lasso1 = ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=10000)
lasso2 = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)
lasso3 = ElasticNet(alpha=0.1, l1_ratio=1, max_iter=10000)
# оцениваем качество модели на кросс-валидации
cv_results1 = cross_validate(lasso1, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results1['train_score'].mean() * 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results1['test_score'].mean() * 100)) 
print('-'*30)
cv_results2 = cross_validate(lasso2, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results2['train_score'].mean() * 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results2['test_score'].mean() * 100))
print('-'*30)
cv_results3 = cross_validate(lasso3, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results3['train_score'].mean() * 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results3['test_score'].mean() * 100)) 

[1.13492457 0.19525842 0.6237965 ]
[1.14379753 0.         0.71993025]
[1.14925373 0.         0.71921642]
------------------------------
MAPE на тренировочных фолдах: 12.80 %
MAPE на валидационных фолдах: 16.24 %
------------------------------
MAPE на тренировочных фолдах: 12.65 %
MAPE на валидационных фолдах: 15.70 %
------------------------------
MAPE на тренировочных фолдах: 12.44 %
MAPE на валидационных фолдах: 16.44 %
